In [134]:
%load_ext autoreload
%autoreload 2
import sys,os
import pickle
from eknot_utils import EventNode,NodeParams
from utils import loadPickle,triHits,norm2,printTopWords,coHits
from eknot_utils import *
from time import time

[resDocInd,tweetsObj,tweetsObjDedup,tweetsScore]= loadPickle('../output/summary_20160207_0213_70_3.pickle')
[Xs,vects,DT,ind2obj] = loadPickle('../data/20160207_0213.pickle')
rootParams,rootNodeDescriptor = loadPickle('../output/plsa_20160207_0213_70.pickle')



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
topK = 100 # ent used
i = 3
window = 7
t0 = time()
Pw_zs = rootNodeDescriptor.Pw_zs
Pe_z = Pw_zs[1][:,i]
evocab = vects[5].get_feature_names()
vocab = vects[0].get_feature_names()

ent_ind,ents = getEntInd(evocab,Pe_z,topK)

print( "entscore in "+str(time() - t0))
t0 = time()

newsObj = [ind2obj[n] for n in resDocInd]
XN,XEn,NEb,sentencesIn,sentencesInObj,ent_text_n = getNewsContext(newsObj,ent_ind,ents,vocab,window)
print( "get news Context in "+str(time() - t0))
print len(newsObj),len(sentencesIn),len(set(sentencesIn))
t0 = time()

XT,XEt,TEb,tweetsIn,tweetsInObj,ent_text_t = getTweetContext(tweetsObjDedup,ent_ind,ents,vocab,window)
print( "get tweet Context in "+str(time() - t0))

print len(tweetsObjDedup),len(tweetsIn),len(set(tweetsIn))

entscore in 0.0376451015472
get news Context in 0.402378082275
11 324 323
append in 0.172278165817
vectorize in 1.0845720768
ents append + vec in 0.266695022583
get tweet Context in 1.5660572052
1580 1580 1580


In [137]:
_XEn = XEn
_XEt = XEt
_NEb = NEb
_TEb = TEb
_XEn.shape,Pe_z.shape

((100, 15042), (4947,))

In [138]:

# #####
# rows = []
# interestedEnts = ['gun_control_other_|','income_tax_other_|','immigration_other_|']
# interestedEnts = ['gun_control_other_|']
# interestedEnts = ents[4:]
# for e in interestedEnts:
#     rows.append(ent_ind[e])
# rows = np.array(rows)
# #rows = np.array([10,33,34,40])
# XEn = _XEn[rows,:]
# XEt = _XEt[rows,:]
# NEb = _NEb[:,rows]
# TEb = _TEb[:,rows]
# #####

XEn = _XEn
XEt = _XEt
NEb = _NEb
TEb = _TEb


t0 = time()

newsScore = XN.dot(Pw_zs[0][:,i])

tweetsScore = XT.dot(Pw_zs[0][:,i])
#tweetsScore = []
#for t in tweetsInObj:
#    tweetsScore.append(float(t.pop))
#tweetsScore = np.array(tweetsScore)

print( "init score in "+str(time() - t0))
t0 = time()

NE_ = XN.dot(XEn.T) #.multiply(NEb)
TE_ = XT.dot(XEt.T) #.multiply(TEb)

##### weight by pop
TE_indptr = TE_.indptr
TE_data = TE_.data
for (XTid,t) in enumerate(tweetsInObj):
    st = TE_indptr[XTid]
    en = TE_indptr[XTid+1]
    TE_data[st:en]*=float(t.pop)
#####

NE,EN = normBypartite(NE_)
TE,ET = normBypartite(TE_)
print( "graph constr in "+str(time() - t0))
t0 = time()

nScore, tScore = triHits(newsScore, tweetsScore, NE, EN, TE, ET, 1, 1, 5)
print( "trihits in "+str(time() - t0))
t0 = time()


init score in 0.000959873199463
graph constr in 0.0190329551697
trihits in 0.00651907920837


In [144]:
nScore, tScore = triHits(newsScore, tweetsScore, NE, EN, TE, ET, 1,1, 20)
#nScore, tScore = triHits(np.ones(newsScore.shape), np.ones(tweetsScore.shape), NE, EN, TE, ET, 1, 1, 50)
print np.linalg.norm(norm2(newsScore)[:10] - nScore[:10])
print "========="
kSummary=5
printSummary(norm2(newsScore),norm2(tweetsScore),sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,kSummary)
print "*****"
printSummary(nScore,tScore,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,kSummary)

for k in range(len(tScore)):
    tScore[k] /= float(tweetsInObj[k].pop)
# print "*****"
# print "*****"
# print "*****"
# print "*****"
# print "*****"
# print "*****"
# printSummary(nScore,tScore,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,kSummary)    

==n converge 7 4.45779175164e-10
0.230181874827
75 0.152733534956 2016-02-11 16:32:40 officer  peter liang convicted in fatal shooting of  akai gurley in  brooklyn | Officer Peter Liang Convicted in Fatal Shooting of Akai Gurley in Brooklyn
209 0.143020557096 2016-02-11 17:28:00 nypd officer  peter liang found guilty of  manslaughter in fatal shooting of  akai gurley in  brooklyn housing development | NYPD Officer Peter Liang found guilty of manslaughter in fatal shooting of Akai Gurley in Brooklyn housing development
135 0.141551367418 2016-02-11 21:17:00 a  jury in  brooklyn found  peter liang guilty in connection with the death of  akai gurley, 28, who was killed by a  bullet fired from liang's gun on nov. 20, 2014, that ricocheted off a wall. | NY policeman guilty of manslaughter in shooting of unarmed black man
22 0.131230335338 2016-02-08 11:37:47 last week, liang's partner,  police officer  shawn  landau, and gurley's girlfriend, melissa  butler, testified at the trial. | Office

In [141]:
from utils import loadPickle,triHits,norm2,printTopWords,coHits
entsScore = XEn.dot(Pw_zs[0][:,i])
#printRankedEnt(entsScore,ents,100,DISPLAY=True)
nScore_tri_n, eScore = coHits(newsScore, entsScore, NE, EN, 1, 1, 20)
printSummary(nScore_tri_n,tScore,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,5)

76 0.00968934990172 2016-02-11 16:32:40 a  new york city  police officer was convicted of  manslaughter on thursday in state  supreme court in  brooklyn for killing an unarmed man who was hit by a  ricocheting  bullet fired from the officer's gun in the stairwell of a  housing project. | Officer Peter Liang Convicted in Fatal Shooting of Akai Gurley in Brooklyn
44 0.00914857214302 2016-02-09 17:15:39 in  closing arguments on tuesday at the  manslaughter trial of a  new york city  police officer charged in the shooting of an unarmed man, the prosecution presented a new argument to  jurors: the officer did not mistakenly fire a  bullet that ricocheted off a wall and killed  akai gurley, who was walking down the stairs of a  brooklyn  housing project, as the defense has argued. | Peter Liang Intentionally Shot Akai Gurley, Prosecution Says
253 0.00875732681102 2016-02-09 13:10:00 assistant district attorney joe  alexis gave his  closing arguments tuesday afternoon at the  brooklyn  mansla

In [142]:
from utils import lexRank
NN = normalize(XN.dot(XN.T),axis=1,norm='l1')
nScore_lex = lexRank(newsScore,NN,passes=200)

printSummary(nScore_lex,tScore,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,10)

==n converge 71 8.87821559623e-10
44 0.00833800240724 2016-02-09 17:15:39 in  closing arguments on tuesday at the  manslaughter trial of a  new york city  police officer charged in the shooting of an unarmed man, the prosecution presented a new argument to  jurors: the officer did not mistakenly fire a  bullet that ricocheted off a wall and killed  akai gurley, who was walking down the stairs of a  brooklyn  housing project, as the defense has argued. | Peter Liang Intentionally Shot Akai Gurley, Prosecution Says
135 0.00781690317757 2016-02-11 21:17:00 a  jury in  brooklyn found  peter liang guilty in connection with the death of  akai gurley, 28, who was killed by a  bullet fired from liang's gun on nov. 20, 2014, that ricocheted off a wall. | NY policeman guilty of manslaughter in shooting of unarmed black man
75 0.0074047787086 2016-02-11 16:32:40 officer  peter liang convicted in fatal shooting of  akai gurley in  brooklyn | Officer Peter Liang Convicted in Fatal Shooting of Akai 

In [143]:
from utils import norm1
NT = normalize(XN.dot(XT.T),axis=1,norm='l1')
TN = normalize(XT.dot(XN.T),axis=1,norm='l1')

nScore_co,tScore_co = coHits(newsScore,tweetsScore,NT,TN,1,1,passes=200)
printSummary(nScore_co,tScore_co,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,10)

==n converge 10 6.22670338555e-10
209 0.020158672505 2016-02-11 17:28:00 nypd officer  peter liang found guilty of  manslaughter in fatal shooting of  akai gurley in  brooklyn housing development | NYPD Officer Peter Liang found guilty of manslaughter in fatal shooting of Akai Gurley in Brooklyn housing development
75 0.0190783308116 2016-02-11 16:32:40 officer  peter liang convicted in fatal shooting of  akai gurley in  brooklyn | Officer Peter Liang Convicted in Fatal Shooting of Akai Gurley in Brooklyn
101 0.013664545135 2016-02-08 10:49:09 nypd officer  peter liang cries on the stand recalling deadly shooting of  akai gurley | NYPD officer Peter Liang cries on the stand recalling deadly shooting of Akai Gurley
2 0.0135736058584 2016-02-08 11:37:47 officer  peter liang, who is accused in the death of  akai gurley, took the stand at his  manslaughter trial monday. | Officer Peter Liang Accused In Death Of Akai Gurley Breaks Down On Witness Stand
210 0.0130405614726 2016-02-11 17:28:0

In [145]:
_res1 = printRankedEnt(NE.T.dot(newsScore),ents,100)
res1 = printRankedEnt(NE.T.dot(nScore),ents,100)
res2 = printRankedEnt(NE.T.dot(nScore_lex),ents,100)
res3 = printRankedEnt(NE.T.dot(nScore_co),ents,100)
res4 = printRankedEnt(NE.T.dot(nScore_tri_n),ents,100)

# _res2 = printRankedEnt(TE.T.dot(tweetsScore),ents,100)
# res2 = printRankedEnt(TE.T.dot(tScore),ents,100)

for e in ents:
    #print res1[e],_res1[e],res2[e],_res2[e],'>|<',res1[e]-_res1[e],res2[e]-_res2[e], res2[e] - res1[e],e
    print res1[e],res2[e],res3[e],res4[e],_res1[e],\
        '\t|',res1[e]-_res1[e],res2[e]-_res1[e],res3[e]-_res1[e],res4[e]-_res1[e],\
        '\t|',res1[e] - res2[e],res3[e] - res2[e],res1[e]-res4[e],res3[e]-res4[e],\
        '\t|',res2[e] - res4[e],e
"""
for e in ents:
    print entscore[e],e
print TE.getrow(1).toarray()    

print NE.getrow(80).toarray()
print NE.getrow(88).toarray()
print NE.getrow(99).toarray()
print NE.getrow(65).toarray()
"""    

0 -4 -4 -1 -4 	| 4 0 0 3 	| 4 0 1 -3 	| -3 new_york_city_police_department_other_|
-4 0 1 -3 0 	| -4 0 1 -3 	| -4 1 -1 4 	| 3 akai_org_|
-5 -5 -7 -5 -7 	| 2 2 0 2 	| 0 -2 0 -2 	| 0 brooklyn_loc_|
-9 -14 -11 -11 -13 	| 4 -1 2 2 	| 5 3 2 0 	| -3 public_housing_other_|
-9 -10 -11 -11 -14 	| 5 4 3 3 	| 1 -1 2 0 	| 1 jury_other_|
-11 0 -8 -6 0 	| -11 0 -8 -6 	| -11 -8 -5 -2 	| 6 bullet_other_|
-5 -9 -2 -6 -7 	| 2 -2 5 1 	| 4 7 1 4 	| -3 manslaughter_other_|
-8 -5 -12 -6 -4 	| -4 -1 -8 -2 	| -3 -7 -2 -6 	| 1 landau_loc_|
6 5 6 5 5 	| 1 0 1 0 	| 1 1 1 1 	| 0 peter_griffin_per_|
-20 -4 -9 -7 -3 	| -17 -1 -6 -4 	| -16 -5 -13 -2 	| 3 butler_other_|
-26 -14 -21 -11 -14 	| -12 0 -7 3 	| -12 -7 -15 -10 	| -3 alexis_davis_per_|
3 0 -1 1 1 	| 2 -1 -2 0 	| 3 -1 2 -2 	| -1 lawyer_other_|
-36 -22 -30 -24 -19 	| -17 -3 -11 -5 	| -14 -8 -12 -6 	| 2 cardiopulmonary_resuscitation_other_|
-7 -7 -12 -5 -9 	| 2 2 -3 4 	| 0 -5 -2 -7 	| -2 closing_argument_other_|
13 14 13 14 14 	| -1 0 -1 0 	| -1 -1 -1 -1 	| 0 

'\nfor e in ents:\n    print entscore[e],e\nprint TE.getrow(1).toarray()    \n\nprint NE.getrow(80).toarray()\nprint NE.getrow(88).toarray()\nprint NE.getrow(99).toarray()\nprint NE.getrow(65).toarray()\n'

In [20]:
from utils import printTopWords
for k in range(topK):
#i=18
    print ents[k]
    printTopWords(k,XEn,vocab,10)
    printTopWords(k,XEt,vocab,10)
    print "+++++++++"

hillary_rodham_clinton_per_|
[(u'clinton', 0.91127212023362503), (u'hillary', 0.32204958980390708), (u'campaign', 0.088028896346187505), (u'sanders', 0.072469148594257862), (u'iowa', 0.044014448173093752), (u'supporters', 0.041057324717920743), (u'new', 0.040631512402912962), (u'secretary', 0.038840102881159647), (u'advantage', 0.036812987445450154), (u'bernie', 0.034380510494772504)]
[(u'hillary', 0.67003435657126253), (u'clinton', 0.62665475629579803), (u'sanders', 0.24558383558974298), (u'bernie', 0.23278738643224145), (u'lead', 0.088128880375277066), (u'attacks', 0.06199925680776424), (u'new', 0.048258634922127071), (u'trump', 0.048211264045453908), (u'iowa', 0.047269446472107908), (u'poll', 0.046948524192749343)]
+++++++++
bernie_sanders_per_|
[(u'bernie', 0.7151885306798762), (u'sanders', 0.5294733581412584), (u'clinton', 0.15905462321174058), (u'hillary', 0.15338414853958637), (u'democratic', 0.1441105750594161), (u'sen', 0.13566020324752465), (u'vermont', 0.13566020324752465), 